In [2]:
import numpy as np 
import gdal
from osgeo import gdal_array

In [118]:
years=['2015', '2017']
months=['01', '02', '03', '04', '05', '06', '07','08', '09', '10', '11', '12']

max_0=0
min_0=0

for i in years:
    for j in months:
        A=gdal_array.LoadFile("/Users/alexandrabjanes/MT/DatosRegion9/Clipped/CW" + i + j + "ws.sdat")
        max_val = np.amax(A)
        min_val = np.amin(A)
        if max_val > max_0:
            max_0 = max_val
            print("global max:", maxtn, i, j)
        if min_val < min_0:
            min_0 = min_val
            print("global min:", mintn, i, j)

print(min_0)
print(max_0)

global max: 83.0 2015 01
global min: -9999.0 2015 01
global max: 83.0 2015 05
global max: 83.0 2015 07
-9999.0
83.0


In [80]:
def GeoTiff1banda(Name, Array, driver, NDV, DataType, bas):
    Array[np.isnan(Array)] = NDV
    driver = gdal.GetDriverByName(driver)
    DataSet = driver.Create(Name, 480, 550, 1, eType=gdal.GDT_Float32)
    Geotr = bas.GetGeoTransform()
    proj = bas.GetProjection()
    DataSet.SetGeoTransform(Geotr)
    DataSet.SetProjection(proj)
    DataSet.GetRasterBand(1).WriteArray(Array)
    DataSet.GetRasterBand(1).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

In [84]:
def normalize(matriz, variable):
    X_norm = np.zeros((550, 480))
    X = matriz
    if variable == 'cwd':
        #maxval = 190
        maxval = 193
        minval = 0
    elif variable == 'NDVI':
        maxval = 9999
        minval = -3000
    elif variable == 'aet':
        #maxval = 133
        maxval = 144
        minval = 0
    elif variable == 'ws':
        #maxval = 61
        maxval = 83
        minval = 0
    elif variable == 'tx':
        maxval = 34
        minval = -1
    elif variable == 'tn':
        #maxval = 15
        maxval = 16
        minval = -9
    elif variable == 'rn':
        maxval = 935
        minval = 0
  
    for i in range(550):
        for j in range(480):
            if X[i,j] < -9990:
                X_norm[i,j] = -99999
            elif X[i,j] > -9990:
                X_norm[i,j] = (X[i,j] - minval) / (maxval - minval)

    return X_norm
    

In [117]:
year=2013
yearmonth=201301
base=gdal.Open("/Users/alexandrabjanes/MT/Datos500m/Normalized/DistanceToUA.tif")

tmax = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "tx.sdat")
tmin = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "tn.sdat")
windspeed = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "ws.sdat")
prec = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "rn.sdat")
aet = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "aet.sdat")
cwd = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "cwd.sdat")
NDVI = gdal_array.LoadFile("/Users/alexandrabjanes/MT/Datos500m/clipped/" +str(year)+ "/C" + str(yearmonth) + "NDVI.sdat")

norm_tmax = normalize(tmax, 'tx')
norm_tmin = normalize(tmin, 'tn')
norm_ws = normalize(windspeed, 'ws')
norm_rn = normalize(prec, 'rn')
norm_aet = normalize(aet, 'aet')
norm_cwd = normalize(cwd, 'cwd')
norm_NDVI = normalize(NDVI, 'NDVI')

GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'tx.tif', norm_tmax, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)
GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'tn.tif', norm_tmin, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)
GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'ws.tif', norm_ws, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)
GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'rn.tif', norm_rn, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)
GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'aet.tif', norm_aet, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)
GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'cwd.tif', norm_cwd, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)
GeoTiff1banda('/Users/alexandrabjanes/MT/Datos500m/Normalized2/'+ str(yearmonth) +'NDVI.tif', norm_NDVI, driver="GTiff", NDV=-99999, DataType=gdal.GDT_Float32, bas=base)

'/Users/alexandrabjanes/MT/Datos500m/Normalized2/201409NDVI.tif'